In [6]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
import numpy as np
import constantes as cons

lista=['EOSUSDT',
'ADAUSDT',
'IOTAUSDT',
'KNCUSDT',
'ZRXUSDT',
'OMGUSDT',
'SXPUSDT',
'ICXUSDT',
'MATICUSDT',
'OCEANUSDT',
'BELUSDT',
'CTKUSDT',
'SANDUSDT',
'MANAUSDT',
'IMXUSDT',
'WOOUSDT',
'MAGICUSDT',
'MINAUSDT',
'AGIXUSDT',
'IDUSDT',
'RDNTUSDT',
'SUIUSDT',
'COMBOUSDT'
]
lista_filtrada = []

#lista=md.lista_de_monedas ()
lista=['XTZUSDT']

for symbol in lista:    
    try:
        #data = md.sigo_variacion_bitcoin(symbol)
        #data = md.estrategia_bb(symbol)
        data = md.estrategia_santa(symbol,tp_flag=True)
        resultado = md.backtesting(data, plot_flag = False)
        if (
            (resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor']))
            and (resultado['Return [%]']/resultado['# Trades']) >=0.33 
            ):
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)} - Win Rate [%]: {resultado['Win Rate [%]']}")        
    except Exception as falla:
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)

In [7]:
resultado

Start                     2023-07-15 03:45:00
End                       2023-07-25 13:30:00
Duration                     10 days 09:45:00
Exposure Time [%]                         1.2
Equity Final [$]                     1010.565
Equity Peak [$]                       1017.24
Return [%]                             1.0565
Buy & Hold Return [%]                 -6.9874
Return (Ann.) [%]                    41.72622
Volatility (Ann.) [%]                8.741527
Sharpe Ratio                         4.773333
Sortino Ratio                             inf
Calmar Ratio                        63.588885
Max. Drawdown [%]                   -0.656187
Avg. Drawdown [%]                   -0.611558
Max. Drawdown Duration        4 days 02:45:00
Avg. Drawdown Duration        2 days 14:45:00
# Trades                                    2
Win Rate [%]                            100.0
Best Trade [%]                        1.12404
Worst Trade [%]                      0.039442
Avg. Trade [%]                    